In [1]:
#Import useful functions
import warnings
warnings.filterwarnings("ignore")
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.utils.multiclass import unique_labels
import itertools
from scipy import misc
import math
import os
import glob
import xlrd
import xlwt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from PIL import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import pylab as plt
import numpy as np
from scipy.signal import argrelextrema
from IPython.display import Image
import matplotlib.patches as patches
import matplotlib as matplotlib
import xlsxwriter
from xlrd import open_workbook
from xlutils.copy import copy
from sklearn.model_selection import train_test_split
from datetime import datetime
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import xlrd

**Specify Inventory File and Sheet Namse, Orders Report's Folder Name, and Revenue or Units Forecast**

In [2]:
invfilename='Dammit Dolls SC Inventory.xlsx'
orderfoldername='Dammit Dolls SC Orders'
sheetname='inventory'
#Type Units Ordered or Ordered Product Sales
forecasttype='Units Ordered'
user='JoyThompson'

In [3]:
inventoryfile= '/Users/'+user+'/Desktop/' + invfilename
orderfolder= '/Users/'+user+'/Desktop/' + orderfoldername
invdata=pd.read_excel(inventoryfile, sheetname=sheetname)

In [4]:
#Transform Inventory Data
inventory=invdata[invdata['detailed-disposition']=='SELLABLE']
inventory=inventory.groupby(['sku','snapshot-date']).count().reset_index()
inventory=inventory.drop(inventory.columns[2:8], axis=1) 
for ind in inventory.index:
    date = inventory.loc[ind,'snapshot-date']
    date=date[0:10]
    date=pd.to_datetime(date,format='%Y/%m/%d')
    inventory.loc[ind,'snapshot-date']=date
    inventory.loc[ind,'Days Sellable in Month']=1
inventory=inventory.groupby(['sku', pd.Grouper(key='snapshot-date', freq='M')])['Days Sellable in Month'].sum().reset_index()

In [5]:
skulist=inventory.groupby(['sku']).sum().reset_index()
skulist=skulist.set_index('sku')

In [6]:
scskus=pd.DataFrame(index=skulist.index)
filepd=pd.DataFrame()
for filename in os.listdir(orderfolder):
    datestamp=filename.split('.xlsx')[0]
    date=pd.to_datetime(datestamp)
    filepd=filepd.append({'Date': date, 'Filename':filename}, ignore_index=True)
filepd=filepd.sort_values('Date')
filepd=filepd.set_index('Date')
   
for date in filepd.index:
    filename=filepd.loc[date, 'Filename']
    filename=orderfolder+ '/'+ filename
    orders=pd.read_excel(filename, sheetname='Sheet1')
    if date not in scskus.columns:
        scskus[date]=''
    for ind in orders.index:
        sku=orders.loc[ind, 'SKU']
        data=orders.loc[ind, forecasttype]
        scskus.loc[sku, date]=data

In [7]:
monthpd=pd.DataFrame({'month':[1,2,3,4,5,6,7,8,9,10,11,12], 'days':[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]})
monthpd=monthpd.set_index('month')

In [11]:
for ind in inventory.index:
    sku=inventory.loc[ind, 'sku']
    date=inventory.loc[ind, 'snapshot-date']
    if date in scskus.columns:
        if sku in scskus.index:
            month=int(str(date)[5:7])
            days=inventory.loc[ind, 'Days Sellable in Month']
            data=scskus.loc[sku,date]
            daysinmon=monthpd.loc[month, 'days']
            unit=scskus.loc[sku, date]
            if not str(unit)=='':
                newunit=float(unit/float(days/daysinmon))
                scskus.loc[sku, date]=newunit

In [10]:
filenamefinal='/Users/JoyThompson/Desktop/scdammitdollskulevel.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')
scskus.to_excel(writer, sheet_name='Sheet1')
writer.save()

PermissionError: [Errno 13] Permission denied: '/Users/JoyThompson/Desktop/scdammitdollskulevel.xlsx'